# **ProofWriterAnalysis**

In [ ]:
import os
DATASET_PATH = "../dataset"

if not os.path.isdir(DATASET_PATH):
    !wget -nc -P ../ https://allenai.org/data/proofwriter
    !unzip -d ../ ../proofwriter
    !mv ../proofwriter-dataset-V2020.12.3 $DATASET_PATH
    !rm ../proofwriter

In [ ]:
from data_loading import *
import torch

TOKENIZER = 

train_dataset = ProofWriterDataset(TOKENIZER, "depth-3/meta-train.jsonl")

In [1]:
import torch 
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline 

torch.random.manual_seed(0) 
model = AutoModelForCausalLM.from_pretrained( 
    "microsoft/Phi-3-mini-128k-instruct",  
    device_map="cuda",  
    torch_dtype="auto",  
    trust_remote_code=True,  
) 

tokenizer = AutoTokenizer.from_pretrained("microsoft/Phi-3-mini-128k-instruct") 

messages = [ 
    {"role": "system", "content": "You are a helpful AI assistant."}, 
    {"role": "user", "content": "Can you provide ways to eat combinations of bananas and dragonfruits?"}, 
    {"role": "assistant", "content": "Sure! Here are some ways to eat bananas and dragonfruits together: 1. Banana and dragonfruit smoothie: Blend bananas and dragonfruits together with some milk and honey. 2. Banana and dragonfruit salad: Mix sliced bananas and dragonfruits together with some lemon juice and honey."}, 
    {"role": "user", "content": "What about solving an 2x + 3 = 7 equation?"}, 
] 

pipe = pipeline( 
    "text-generation", 
    model=model, 
    tokenizer=tokenizer, 
) 

generation_args = { 
    "max_new_tokens": 500, 
    "return_full_text": False, 
    "temperature": 0.0, 
    "do_sample": False, 
} 

output = pipe(messages, **generation_args) 
print(output[0]['generated_text'])


OSError: /home/ivan/Desktop/NLP/.venv/lib/python3.13/site-packages/torch/lib/libtorch_global_deps.so: cannot open shared object file: No such file or directory